# Leer base de datos

Archivo para probar la lectura de los datos de [UC Irvine ML Repository](https://archive.ics.uci.edu/dataset/109/wine).

In [1]:
import numpy as np
import pandas as pd
from myqp import *

In [2]:
df = pd.read_csv('wine.data')

## Función a minimizar

El objetivo es minimizar 
$$\frac{1}{2} w^{\tau}w$$ 
sujeto a 
$$ 
\begin{align*}
w^{\tau}x_i + \beta &\leq -1, \quad i = 1, 2, \ldots, s \\
w^{\tau}x_i + \beta &\geq 1, \quad i = s+1, s+2, \ldots, s+r, \\
\end{align*}
$$
con las variables $w \in \mathbb{R}^n$, $\beta \in \mathbb{R}$

In [3]:
A = df[df['class'] == 1]
B = df[df['class'] == 2]
C = df[df['class'] == 3]

In [4]:
X_A = A.drop(columns=['class'])
y_A = A['class']
X_B = B.drop(columns=['class'])
y_B = B['class']
X_C = C.drop(columns=['class'])
y_C = C['class']

In [5]:
X_A.shape

(59, 13)

In [6]:
# s y r son el número de filas de A y B, respectivamente
# n es el número de columnas que tienen A y B y es el tamaño de nuestra w
(s, n) = X_A.shape
r = X_B.shape[0]

In [7]:
Q = np.eye(n)
c = np.zeros(n)
F = np.concatenate((-X_A, X_B))
beta = 10
d = np.concatenate((np.ones(s) + beta, np.ones(r) - beta))

---

$$\frac{1}{2} w^{\tau}w$$ 
sujeto a 
$$ 
\begin{align*}
-w^{\tau}x_i &\geq 1 + \beta, \quad i = 1, 2, \ldots, s \\
w^{\tau}x_i &\geq 1 - \beta, \quad i = s+1, s+2, \ldots, s+r, \\
\end{align*}
$$

In [8]:
w,mu,z,iter = myqp_intpoint_proy(Q, F, c, d, verbose=False)

In [9]:
# valor de la función objetivo
np.dot(w,w)

39.44142062257272

In [10]:
# Todos tendrían que ser <= -1
np.dot(X_A, w) + beta

array([ -7.25226783,  -7.92698534, -11.22344886, -17.32449515,
        -1.00000005, -16.0025566 , -11.94967121, -11.2864576 ,
        -6.21073482,  -7.42782356, -14.607566  ,  -9.35120243,
       -12.0425962 , -10.59905352, -19.15215316, -12.87651496,
       -11.33440669,  -8.28022011, -20.96388513,  -6.34324587,
        -1.00000027,  -3.73167032,  -5.49498479,  -4.95800683,
        -1.00000062,  -1.00000002, -11.32966915,  -8.90822989,
        -3.39244637,  -4.70040728,  -9.49367399, -15.33163877,
        -3.58166799,  -8.32789458,  -6.60134758,  -2.97297289,
        -5.16606237,  -6.82581464,  -4.08906978,  -6.05815827,
        -2.66265361,  -7.35292831, -10.83106896,  -2.13470107,
        -3.0485042 ,  -9.41070034,  -8.71538117,  -6.05641206,
        -6.49361894, -12.22549694, -10.75253491, -11.02261421,
       -12.20535594, -13.72699589,  -7.1121592 ,  -6.8707921 ,
        -5.3292317 , -13.24323968, -13.41271366])

In [11]:
# Todos tendrían que ser >= 1
np.dot(X_B, w) + beta

array([ 9.59073566,  4.6286704 ,  7.18597144,  6.82601296,  8.27744825,
       10.51926929,  1.35461254,  6.03517664,  7.82781294,  3.34705972,
        7.36566973,  1.0000003 , 10.48507559,  9.27694263,  2.43271049,
        1.00000006,  8.11514819, 10.33454252,  5.43929735,  3.16412963,
        6.67917917, 14.02934742,  1.92563511,  7.84702425,  3.75961198,
        4.09917927,  9.51006017,  9.97076069,  8.72256883,  2.96329952,
        7.66858717,  7.08397128,  8.75521099,  7.78154517,  9.59435791,
        8.77700688,  1.00000012,  4.32169883,  8.37351128,  2.13893381,
        7.39310706,  4.2904442 ,  8.05423062,  6.20960508, 10.52865475,
        5.36105542,  9.88990774,  6.04856203,  7.46602077, 11.63855583,
        1.00000014,  3.76714091,  9.99554551,  1.00000003,  8.85663287,
        8.865131  , 12.71346939,  9.33339273, 12.2240543 ,  5.76227081,
        5.14001884,  1.00000026,  1.00000017,  7.25424396,  3.92043064,
        3.58411914,  8.02676561,  7.27317094,  8.00781865, 11.26

---

## Meter $\beta$ en el vector de variables

$$\frac{1}{2} w^{\tau}w$$ 
sujeto a 
$$ 
\begin{align*}
-w^{\tau}x_i - \beta &\geq 1, \quad i = 1, 2, \ldots, s \\
w^{\tau}x_i + \beta &\geq 1, \quad i = s+1, s+2, \ldots, s+r, \\
\end{align*}
$$

In [12]:
# s y r son el número de filas de A y B, respectivamente
# n es el número de columnas que tienen A y B y es el tamaño de nuestra w
(s, n) = X_A.shape
r = X_B.shape[0]

In [13]:
# Agregamos la variable de la beta
n += 1

In [14]:
Q = np.eye(n)
c = np.zeros(n)
F = np.concatenate((-X_A, X_B))
aux = np.concatenate((-np.ones(s),np.ones(r)))
F = np.c_[F, aux]
d = np.ones(s+r)

In [15]:
w_hat,mu,z,iter = myqp_intpoint_proy(Q, F, c, d, verbose=False)

In [16]:
# Valor objetivo REAL - contando a la beta
np.dot(w_hat,w_hat)

119.52546373358149

In [17]:
w = w_hat[:-1]
beta = w_hat[-1]

In [18]:
# valor de la función objetivo sin tomar en cuenta a beta
np.dot(w,w)

78.311901266059

In [19]:
beta

6.419779004570367

In [20]:
# Todos tendrían que ser <= -1
np.dot(X_A, w) + beta

array([-10.07552643, -12.04606895, -16.29746866, -24.86151492,
        -1.00000003, -23.4011069 , -16.90208806, -16.18358311,
        -8.54724189, -10.63553622, -21.33579444, -13.59952807,
       -17.66335019, -15.13299891, -27.91944975, -18.96302721,
       -16.21235286, -12.10991678, -30.99233347,  -8.91816492,
        -1.0000003 ,  -4.93615165,  -7.81815207,  -7.37311012,
        -1.00000022,  -1.00000002, -16.62911581, -13.36167046,
        -4.4603643 ,  -6.34740157, -13.7304612 , -22.72119623,
        -5.17578656, -12.18803203,  -9.61548708,  -4.07560221,
        -7.60340337, -10.30500014,  -6.59235214,  -7.9411302 ,
        -3.61912676, -10.55413701, -15.62395505,  -2.65651842,
        -4.54603736, -12.94908844, -11.97253879,  -8.63641431,
        -9.00260988, -17.87166607, -16.16847761, -15.94425151,
       -17.87310386, -20.03252331, -10.37523424,  -9.72799696,
        -7.25271742, -19.63178698, -19.73306973])

In [21]:
# Todos tendrían que ser >= 1
np.dot(X_B, w) + beta

array([13.14531717,  6.03340411, 10.15452095,  9.99840225, 11.77399114,
       14.91022512,  1.53159336,  8.69862113, 11.20078497,  4.80726588,
       10.19056221,  1.00000021, 16.23837678, 14.14614345,  3.68639521,
        1.00000005, 10.95411088, 14.87976283,  7.55208543,  3.96828833,
       10.2325905 , 19.97435176,  2.64053115, 11.23994891,  6.20530767,
        5.77410955, 13.7936127 , 14.26248709, 12.52500846,  3.92331896,
       11.14651793, 10.10331699, 12.79451757, 11.46512629, 14.16872379,
       12.33342567,  1.0000001 ,  6.08581599, 11.73047908,  2.44344413,
       10.70967187,  5.45850252, 11.57576336,  9.33553246, 14.96555927,
        7.78853961, 14.82197237,  8.66068063, 11.10947068, 16.9765514 ,
        1.0000001 ,  5.34537753, 14.70368951,  1.00000003, 12.25544937,
       12.8898851 , 17.63189542, 13.35454913, 17.82470159,  8.67952065,
        7.46484027,  1.00000014,  1.00000015, 11.37497974,  6.71451522,
        5.68575412, 11.57690141, 10.62342975, 11.726669  , 16.58